<a href="https://colab.research.google.com/github/Mqondisi-Mavuso/cow_facial_recognition/blob/main/Copy_of_cattle_id_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import IPython.display as ipd
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import csv
import os

# Define the directory to search for JPG files
directory = "/content/drive/MyDrive/DATA/train"

# Create a CSV file to write the results to
with open("labels.csv", "w", newline="") as csvfile:
    fieldnames = ["filename", "id"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()

    # Loop through all subdirectories in the main directory
    for subdir, dirs, files in os.walk(directory):
        # Loop through all JPG files in each subdirectory
        for file in files:
            if file.lower().endswith(".jpg"):
                # Get the filename and ID
                filename = file
                id = file[3:file.index("_")]

                # Write the filename and ID to the CSV file
                writer.writerow({"filename": filename, "id": id})


In [ ]:
import os

def rename_images(root_dir):
    """
    Rename every image in every subdirectory using its directory name followed by its unique id.
    """
    # Traverse all directories and files in the root directory
    for subdir, dirs, files in os.walk(root_dir):
        # For each image file in the current directory
        for i, file in enumerate(files):
            # Check if the file is an image
            if file.endswith(".jpg") or file.endswith(".jpeg") or file.endswith(".png") or file.endswith(".gif"):
                # Get the directory name and use it as the prefix for the new filename
                prefix = os.path.basename(subdir)
                # Generate a unique id for the file
                unique_id = str(i+1)
                # Get the file extension
                ext = os.path.splitext(file)[1]
                # Generate the new filename
                new_filename = prefix + "_" + unique_id + ext
                # Rename the file
                os.rename(os.path.join(subdir, file), os.path.join(subdir, new_filename))

# Example usage
root_dir = "/content/drive/MyDrive/DATA/train"
rename_images(root_dir)


In [ ]:
import os
import random

# Set the path to the directory containing the subdirectories of images
parent_directory = "/content/drive/MyDrive/DATA/train"

# Set the number of images to keep in each subdirectory
num_images_to_keep = 20

# Loop through each subdirectory in the parent directory
for subdir in os.listdir(parent_directory):
    subdir_path = os.path.join(parent_directory, subdir)
    if os.path.isdir(subdir_path):
        # Get a list of all the images in the subdirectory
        images = [os.path.join(subdir_path, f) for f in os.listdir(subdir_path) if f.endswith(".jpg") or f.endswith(".png")]
        # If there are more than num_images_to_keep images, randomly select which ones to keep
        if len(images) > num_images_to_keep:
            images_to_keep = random.sample(images, num_images_to_keep)
            # Delete all the images that were not selected to be kept
            for image in images:
                if image not in images_to_keep:
                    os.remove(image)


In [ ]:
# import the videos to read each frame 
cap = cv2.VideoCapture("DATA/test/c22/IMG_3751.MOV")

In [ ]:
# total number of frames
num_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

In [ ]:
#getting the frames from each video
def get_frames(num_frame):
    for frame in range(num_frame//2):
        ret, img = cap.read()
        # this rotates the image before writing to file
        rotated_img = cv2.rotate(img, cv2.ROTATE_180)        
        if ret == False:
            break
        else:
            path = r"DATA/test/c22"
            name = f'/b_c{frame + 1}.jpg' 
            # write the file
            cv2.imwrite(f'{path}{name}', rotated_img)

In [ ]:
# this call will write the images to our training folder 
get_frames(num_frame=num_frame)
cap.release()

In [ ]:
#!pip install tensorflow-gpu

In [ ]:
# This is for downloading the YOLOv8 cow face images and labels
# !pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="my_api_here")                    
# project = rf.workspace("face-cow").project("thecowface")
# dataset = project.version(1).download("yolov8")

#!pip install roboflow

# from roboflow import Roboflow
# rf = Roboflow(api_key="GzhyGHnKKy58hjhTCDdG")
# project = rf.workspace("cowseg").project("thecowface-tj1no")
# dataset = project.version(1).download("coco-segmentation")


In [ ]:
#training the model to segment the cow face using custom dataset 
%pip install ultralytics
from ultralytics import YOLO

model = YOLO("/content/drive/MyDrive/Colab_Notebooks/runs/detect/train/weights/best.pt")
# Training the model
#!yolo train model=yolov8n-seg.pt data=coco128.yaml epochs=3 imgsz=640
#model.train(data="/content/drive/MyDrive/cow_id_AI/cow_facial_recognition/Thecowface-1/data.yaml", epochs=100)

In [ ]:
def show_img(image_path):
  img = cv2.imread(image_path)
  fig, ax = plt.subplots(figsize=(15,15))
  plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  ax.grid(False)           # remove the grid lines
  plt.show()

In [ ]:
# image_path = "/content/runs/detect/train/val_batch0_pred.jpg"
# show_img(image_path=image_path)

In [ ]:
# Load the first image
confusion_matrix = cv2.imread("/content/runs/detect/train/confusion_matrix.png")

# Load the second image
results = cv2.imread("/content/runs/detect/train/results.png")

# Create a figure with two subplots
fig, ax = plt.subplots(1, 2, figsize=(20,20))

# Display the first image on the first subplot
ax[0].imshow(cv2.cvtColor(confusion_matrix, cv2.COLOR_BGR2RGB))

# Display the second image on the second subplot
ax[1].imshow(cv2.cvtColor(results, cv2.COLOR_BGR2RGB))

ax[0].grid(False)
ax[1].grid(False)    

# Show the figure
plt.show()


In [ ]:
#validate the model on images it has not seen

#model = YOLO("/content/runs/detect/train2/weights/best.pt")  # load a custom model

# Validate the model
#!yolo task=detect mode=val model="/content/runs/detect/train2/weights/best.pt" data=data.yaml
#!yolo val model='/content/runs/detect/train2/weights/best.pt data=coco128'

# import subprocess

# # Set up the command
# command = ['yolo', 'task=detect', 'mode=predict', 'model=/content/runs/detect/train/weights/best.pt', 'conf=0.25', 'source=/content/drive/MyDrive/DATA/test/c12/b_c13.jpg', 'save=True']

# # Run the command using subprocess
# result = subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

# # Print the output
# print(result.stdout)



This is for saving the cropped facial images after facial detection

In [ ]:
from os import listdir, path
import cv2
import pandas as pd

the_path = "/content/drive/MyDrive/DATA/train/cow10"
image_filenames = listdir(the_path)

for filename in image_filenames:
    image_path = path.join(the_path, filename)
    image = cv2.imread(image_path)
    results = model(image)  # predict on an image
    a = results[0].boxes.boxes
    px = pd.DataFrame(a).astype("float")

    for index, row in px.iterrows():
        x1 = int(row[0])
        y1 = int(row[1])
        x2 = int(row[2])
        y2 = int(row[3])
        
        if (x1 != 0) and (x2 != 0):
            # Crop the object from the image
            object_image = image[y1:y2, x1:x2]
            
            # Save the cropped image as a new file in the same directory as the original image
            new_filename = f"{filename[:-4]}_cropped_{index}.jpg"
            cv2.imwrite(path.join(the_path, new_filename), object_image)
        else:
          continue


In [ ]:
# removing the uncropped images
import os

folder_path = "/content/drive/MyDrive/DATA/train/c9"

for file_name in os.listdir(folder_path):
    if file_name.endswith(".jpg") and "cr" not in file_name:
        os.remove(os.path.join(folder_path, file_name))


In [ ]:
################################## Cow Face Segmentation Above ######################### 

In [ ]:
################################## Cow Face Identification below ######################### 

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
image_gen = ImageDataGenerator(rotation_range=30,
                               width_shift_range=0.1,
                               height_shift_range=0.1,
                               rescale=1/255,
                               shear_range=0.2,
                               zoom_range=0.2,
                               horizontal_flip=True,
                               fill_mode="nearest"                                
                                )

In [ ]:
#image_gen.flow_from_directory("DATA/train/")

This is for creating pairs of facial images and their corresponding labels for training and validation in the Siamese Neural Network model for facial recognition

In [ ]:
import tensorflow as tf
import numpy as np
import os

# Define a generator function that yields batches of data
def data_generator():
    while True:
        x = np.random.rand(32, 224, 224, 3).astype(np.float32)
        y = np.random.randint(0, 10, (32,), np.int32)
        yield (x, x), y

# Create a dataset from the generator
train_dataset = tf.data.Dataset.from_generator(data_generator,
                                                output_types=((tf.float32, tf.float32), tf.int32),
                                                output_shapes=((None, 224, 224, 3), (None,)))

# Define the model
inputs = tf.keras.layers.Input(shape=(224, 224, 3))
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(inputs)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Conv2D(64, 3, activation='relu')(x)
x = tf.keras.layers.MaxPooling2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(10, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

# Train the model
model.fit(train_dataset, epochs=5, steps_per_epoch=100)


Creating the Siamese Neural Network model

In [ ]:
# Define the Siamese neural network model
def create_siamese_model():
    # Define the input shapes for the image pairs
    input_shape = (224, 224, 3)
    input_1 = tf.keras.layers.Input(shape=input_shape)
    input_2 = tf.keras.layers.Input(shape=input_shape)

    # Define the convolutional base using MobileNetV2
    base_model = tf.keras.applications.MobileNetV2(weights='imagenet',
                                                   include_top=False,
                                                   input_shape=input_shape)

    # Freeze the convolutional base so that its weights are not updated during training
    base_model.trainable = False

    # Define the feature extraction layers
    x1 = base_model(input_1)
    x1 = tf
        x1 = tf.keras.layers.GlobalAveragePooling2D()(x1)
    
    x2 = base_model(input_2)
    x2 = tf.keras.layers.GlobalAveragePooling2D()(x2)

    # Define the distance metric layer
    distance = tf.keras.layers.Lambda(lambda x: tf.keras.backend.abs(x[0] - x[1]))([x1, x2])

    # Define the classification layers
    fc1 = tf.keras.layers.Dense(512, activation='relu')(distance)
    fc2 = tf.keras.layers.Dense(128, activation='relu')(fc1)
    output = tf.keras.layers.Dense(len(label_indices), activation='softmax')(fc2)

    # Define the Siamese model
    model = tf.keras.models.Model(inputs=[input_1, input_2], outputs=output)

    # Compile the model
    optimizer = tf.keras.optimizers.Adam(lr=0.001)
    model.compile(optimizer=optimizer, loss='sparse_categorical_crossentropy', metrics=['accuracy'])

    return model

In [ ]:
# Create an instance of the Siamese neural network model
siamese_model = create_siamese_model()

# Train the model
siamese_model.fit(train_pairs, train_labels, batch_size=32, epochs=10, validation_data=(val_pairs, val_labels))

Evaluating the model

In [ ]:
# Load the test dataset
test_pairs, test_labels = create_pairs(test_data, label_indices)

# Evaluate the model on the test dataset
loss, accuracy = siamese_model.evaluate(test_pairs, test_labels)

print('Test loss:', loss)
print('Test accuracy:', accuracy)
